### Exploring the WMS with OWSLib

In [1]:
from owslib.wms import WebMapService


url = 'http://oos.soest.hawaii.edu/thredds/wms/hioos/satellite/dhw_5km'

web_map_services = WebMapService(url)

print('\n'.join(web_map_services.contents.keys()))

surface_flag
CRW_SST
CRW_BAA
CRW_HOTSPOT
CRW_DHW
CRW_SSTANOMALY
CRW_SEAICE


### Layer metadata

In [2]:
layer = 'CRW_SST'
wms = web_map_services.contents[layer]

name = wms.title

lon = (wms.boundingBox[0] + wms.boundingBox[2]) / 2.
lat = (wms.boundingBox[1] + wms.boundingBox[3]) / 2.
center = lat, lon

time_interval = '{0}/{1}'.format(
    wms.timepositions[0].strip(),
    wms.timepositions[-1].strip()
)
style = 'boxfill/sst_36'

if style not in wms.styles:
    style = None

In [6]:
time_interval

'1985-03-25T12:00:00.000Z/2019-08-21T12:00:00.000Z'

### Single layer

In [4]:
import folium
from folium import plugins

lon, lat = -50, -40

m = folium.Map(
    location=[lat, lon],
    zoom_start=5,
    control_scale=True
)

w = folium.raster_layers.WmsTileLayer(
    url=url,
    name=name,
    styles=style,
    fmt='image/png',
    transparent=True,
    layers=layer,
    overlay=True,
    COLORSCALERANGE='1.2,28',
)

w.add_to(m)

time = plugins.TimestampedWmsTileLayers(
    w,
    period='PT1H',
    time_interval=time_interval
)

time.add_to(m)

folium.LayerControl().add_to(m)

m

### Multiple layers

In [5]:
m = folium.Map(
    location=[lat, lon],
    zoom_start=5,
    control_scale=True
)

w0 = folium.raster_layers.WmsTileLayer(
    url=url,
    name='sea_surface_temperature',
    styles=style,
    fmt='image/png',
    transparent=True,
    layers='CRW_SST',
    overlay=True,
)

w1 = folium.raster_layers.WmsTileLayer(
    url=url,
    name='analysed sea surface temperature anomaly',
    styles=style,
    fmt='image/png',
    transparent=True,
    layers='CRW_SSTANOMALY',
    overlay=True,
)

w0.add_to(m)
w1.add_to(m)

time = folium.plugins.TimestampedWmsTileLayers(
    [w0, w1],
    period='PT1H',
    time_interval=time_interval
)

time.add_to(m)

folium.LayerControl().add_to(m)

m